### Building a RAG System with LangChain and FAISS 
Introduction to RAG (Retrieval-Augmented Generation)
RAG combines the power of retrieval systems with generative AI models. Instead of relying solely on the model's training data, RAG:

1. Retrieves relevant documents from a knowledge base
2. Uses these documents as context for the LLM
3. Generates responses based on both the retrieved context and the model's knowledge

### FAISS 
https://github.com/facebookresearch/faiss

FAISS is a library for efficient similarity search and clustering of dense vectors.

Key advantages:
1. Extremely fast similarity search
2. Memory efficient
3. Supports GPU acceleration
4. Can handle millions of vectors

How it works:
- Indexes vectors for fast nearest neighbor search
- Returns most similar vectors based on distance metrics

In [1]:
## load libraries
import os
from dotenv import load_dotenv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# LangChain core imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough, 
 
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# LangChain specific imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Load environment variables
load_dotenv()

True

### Data Ingestion And Processing

In [2]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolu

In [3]:
## text splitting
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "]
)

## split the documents into chunks
chunks = text_splitter.split_documents(sample_documents)
print(chunks[0])
print(chunks[1])


page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.' metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}
page_content='Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.' metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}


In [4]:
print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 4 chunks from 4 documents

Example chunk:
Content: Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
Metadata: {'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}


In [5]:
### load the embedding models
import os
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [6]:
# Initialize OpenAI embeddings with the latest model

embeddings=OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536
)

## Example: create a embedding for a single text
sample_text="What is machine learning"
sample_embedding=embeddings.embed_query(sample_text)
sample_embedding

[-0.0059153237380087376,
 -0.005926128011196852,
 0.0005067865713499486,
 -0.033579591661691666,
 0.021208731457591057,
 0.022148700430989265,
 -0.0008082925342023373,
 0.009243031032383442,
 -0.022580871358513832,
 0.0378364622592926,
 0.015212375670671463,
 -0.03669121488928795,
 -0.033968545496463776,
 0.0015855228994041681,
 0.026772918179631233,
 0.015957869589328766,
 0.0008049161988310516,
 -0.03087853081524372,
 0.023358775302767754,
 0.004867312032729387,
 9.993925777962431e-05,
 0.03247755765914917,
 0.042287811636924744,
 -0.02930111065506935,
 0.015957869589328766,
 -0.020971037447452545,
 0.02666487544775009,
 0.012360055930912495,
 0.007400907576084137,
 -0.00594773655757308,
 -0.009415898472070694,
 -0.02906341664493084,
 -0.03128909319639206,
 0.027572430670261383,
 0.041725993156433105,
 0.00045951802167110145,
 -0.008130193687975407,
 -0.0106583870947361,
 -0.05557703226804733,
 0.002597070299088955,
 -0.05626850575208664,
 -0.016249584034085274,
 0.030208667740225792

In [7]:
texts=["AI","MAchine learning","Deep Learning","Neural Network"]
batch_embeddings=embeddings.embed_documents(texts)
print(batch_embeddings[0])

[-0.008167067542672157, -0.024646570906043053, 0.002890851115807891, 0.0251751821488142, 0.006528367754071951, -0.02828078344464302, -0.005031732842326164, 0.020985927432775497, -0.03687074035406113, 0.012865113094449043, -0.0030659539625048637, -0.020153362303972244, 0.000287433183984831, -0.03274755924940109, 0.006425948813557625, -0.025307336822152138, -0.031055999919772148, -0.054394252598285675, 0.03277399018406868, -0.018369292840361595, 0.01663808710873127, 0.04831520840525627, -0.024937307462096214, 0.014351836405694485, 0.029364438727498055, 0.0040901415050029755, 0.009323407895863056, 0.01335407979786396, 0.0024894357193261385, -0.022584980353713036, 0.032113224267959595, -0.028016475960612297, 0.005368723534047604, -0.0381922721862793, -0.01670416258275509, 0.014365051873028278, -0.038615163415670395, -0.010387240909039974, -0.010532609187066555, -0.019188642501831055, 0.03203393518924713, 0.014682219363749027, -0.021514538675546646, 0.016083043068647385, -0.0118144955486059

In [8]:
print(batch_embeddings[1])

[-0.0182332843542099, 0.010754693299531937, 0.01752162165939808, -0.03528864309191704, 0.03744817152619362, 0.0260861124843359, 0.015914246439933777, 0.008398844860494137, -0.010932608507573605, 0.03489600494503975, -0.006607418414205313, -0.06866316497325897, -0.03239291533827782, 0.008987806737422943, 0.03658926859498024, 0.01869954541325569, 0.01629461720585823, -0.01539890468120575, 0.020625943318009377, 0.012993975542485714, -0.0026334580034017563, 0.03779173269867897, 0.03408617898821831, -0.030012525618076324, 0.020675022155046463, 0.007920312695205212, 0.03612300753593445, 0.0376935712993145, 0.010539967566728592, -0.03484692424535751, -0.010883528739213943, -0.024221066385507584, -0.01395103894174099, -0.013840609230101109, 0.01196329202502966, -0.001851242734119296, -0.02763213776051998, 0.027828458696603775, -0.045080140233039856, -0.00519022811204195, -0.01974250003695488, -0.04277336969971657, 0.034356120973825455, 0.06827051937580109, -0.020589131861925125, -0.03484692424

In [9]:
### Compare Embedding using cosine similarity

def compare_embeddings(text1:str,text2:str):
    """Compare semantic simialrity of 2 texts usign embeddings"""

    emb1=np.array(embeddings.embed_query(text1))
    emb2=np.array(embeddings.embed_query(text2))

    ## Calculate the simialrity score

    similarity=np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity

In [10]:
# Test semantic similarity
print("\nSemantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")


Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.563


In [11]:
print(f"'AI' vs 'Pizza': {compare_embeddings('AI', 'Pizza'):.3f}")

'AI' vs 'Pizza': 0.254


In [12]:
print(f"'Machine Learning' vs 'ML': {compare_embeddings('Machine Learning', 'ML'):.3f}")

'Machine Learning' vs 'ML': 0.461


### Create FAISS Vector Store

In [13]:
vectorstore=FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 4 vectors


In [14]:
vectorstore

In [15]:
## Save vector tore for later use
vectorstore.save_local("faiss_index")
print("Vector store saved to 'faiss_index' directory")

Vector store saved to 'faiss_index' directory


In [16]:
## load vector store
loaded_vectorstore=FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 4 vectors


In [17]:
## Similarity Search 
query="What is deep learning"

results=vectorstore.similarity_search(query,k=3)
print(results)

[Document(id='a96df7ce-219c-4b3d-96cf-1ef3fd501031', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'), Document(id='c8a7887f-b2b5-449f-9f0f-0b1fc501bf76', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), Document(id='0c54115b-2574-44a1-b2b9-312d5dcbc649', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human lang

In [18]:
print(f"Query: {query}\n")
print("Top 3 similar chunks:")
for i, doc in enumerate(results):
    print(f"\n{i+1}. Source: {doc.metadata['source']}")
    print(f"   Content: {doc.page_content[:200]}...")

Query: What is deep learning

Top 3 similar chunks:

1. Source: Deep Learning
   Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning ...

2. Source: ML Basics
   Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised...

3. Source: NLP Overview
   Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
      ...


In [19]:
### Similarity Search with score
results_with_scores=vectorstore.similarity_search_with_score(query,k=3)

print("\n\nSimilarity search with scores:")
for doc, score in results_with_scores:
    print(f"\nScore: {score:.3f}")
    print(f"Source: {doc.metadata['source']}")
    print(f"Content preview: {doc.page_content[:100]}...")



Similarity search with scores:

Score: 0.556
Source: Deep Learning
Content preview: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses m...

Score: 1.208
Source: ML Basics
Content preview: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being...

Score: 1.274
Source: NLP Overview
Content preview: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
...


In [20]:
chunks

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.'),
 Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'),
 Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recogn

In [21]:
### Search with metadata filtering
filter_dict={"topic":"ML"}
filtered_results=vectorstore.similarity_search(
    query,
    k=3,
    filter=filter_dict
)
print(filtered_results)

[Document(id='c8a7887f-b2b5-449f-9f0f-0b1fc501bf76', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.')]


In [22]:
len(filtered_results)

1

### Build RAG Chain with LCEL

In [24]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("openai:gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x111ee8ef0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f7b9550>, root_client=<openai.OpenAI object at 0x111e99c10>, root_async_client=<openai.AsyncOpenAI object at 0x111ee8320>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [25]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C6NEZmexhwLVmh7YHYfBQkjZwkVRn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--335fc3e9-7bc1-4b17-b2d6-62556c2d494a-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:""")

In [28]:
## Basic retriever
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1100f2ba0>, search_kwargs={'k': 3})

In [29]:
from typing import List
# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [30]:
simple_rag_chain=(
    {"context":retriever | format_docs,"question":RunnablePassthrough() }
    | simple_prompt
    | llm
    |StrOutputParser()
)

In [31]:
### Conversational RAg Chain
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}"),
])

In [33]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()
conversational_rag

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_docs(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag=

In [34]:
### streaming RAG chain
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
)

print("Modern RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

Modern RAG chains created successfully!
Available chains:
- simple_rag_chain: Basic Q&A
- conversational_rag: Maintains conversation history
- streaming_rag_chain: Supports token streaming


In [35]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)
    
    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print()

In [37]:
test_rag_chains("What is the difference between AI and machine learning")

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: The difference between AI and machine learning is that AI is the simulation of human intelligence in machines, while machine learning is a subset of AI that enables systems to learn from data.

2. Streaming RAG:
Answer: AI is the simulation of human intelligence in machines, while machine learning is a subset of AI that enables systems to learn from data.


In [38]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: AI is the simulation of human intelligence in machines, while Machine Learning is a subset of AI that enables systems to learn from data.

2. Streaming RAG:
Answer: Artificial Intelligence (AI) is the simulation of human intelligence in machines, designed to think like humans and mimic their actions, while Machine Learning is a subset of AI that enables systems to learn from data by finding patterns in data without being explicitly programmed.


Question: Explain deep learning in simple terms

1. Simple RAG Chain:
Answer: Deep learning is a type of machine learning that uses artificial neural networks with multiple layers to help computers learn and understand complex patterns in data. It has significantly improved computer vision, natural language processing, and speech recognition technologies.

2. Streaming RAG:
Answer: Deep learning is a type of machine learning that uses artificial ne

In [39]:
## Conversational example
print("\n3. Conversational RAG Example:")
chat_history = []

# First question
q1 = "What is machine learning?"
a1 = conversational_rag.invoke({
    "input": q1,
    "chat_history": chat_history
})

print(f"Q1: {q1}")
print(f"A1: {a1}")


3. Conversational RAG Example:
Q1: What is machine learning?
A1: Machine Learning is a subset of AI that enables systems to learn from data. Instead of being explicitly programmed, ML algorithms find patterns in data. Common types include supervised, unsupervised, and reinforcement learning.


In [40]:
# Update history
chat_history.extend([
    HumanMessage(content=q1),
    AIMessage(content=a1)
])

In [41]:
# Follow-up question
q2 = "How is it different from traditional programming?"
a2 = conversational_rag.invoke({
    "input": q2,
    "chat_history": chat_history
})
print(f"\nQ2: {q2}")
print(f"A2: {a2}")


Q2: How is it different from traditional programming?
A2: Machine learning differs from traditional programming in that traditional programming involves explicit instructions provided by a programmer to perform a specific task. In contrast, machine learning algorithms learn from data to make predictions or decisions without being explicitly programmed for every scenario. The system improves its performance over time by learning patterns and relationships in data.
